# **NEURAL NETWORK**

In [ ]:
# Improved Deep Neural Network for HAR Classification

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Load dataset
df = pd.read_csv("SecureCare_Data.csv")

# Drop unnecessary columns
df = df.drop(['Time', 'Person'], axis=1)

# Encode target
label_encoder = LabelEncoder()
df['Class'] = label_encoder.fit_transform(df['Class'])

# Split features and labels
X = df.drop('Class', axis=1).values
y = df['Class'].values

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# One-hot encode labels
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Build deeper model
model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.4),

    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),

    Dense(64),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(y_train_cat.shape[1], activation='softmax')
])

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, verbose=1)

# Train model
history = model.fit(
    X_train, y_train_cat,
    epochs=60,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Evaluate
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("📊 Model Performance:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [ ]:
# ===============================
# Federated Learning with 5 Clients (FedAvg)
# ===============================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# -------------------------------
# Load and preprocess dataset
# -------------------------------
df = pd.read_csv("/content/SecureCare_Data.csv")
df = df.drop(['Time', 'Person'], axis=1)

label_encoder = LabelEncoder()
df['Class'] = label_encoder.fit_transform(df['Class'])

X = df.drop('Class', axis=1).values
y = df['Class'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# -------------------------------
# Split data for 5 clients
# -------------------------------
num_clients = 5
client_data = []
split_size = len(X_train) // num_clients
for i in range(num_clients):
    start, end = i * split_size, (i + 1) * split_size
    client_data.append((X_train[start:end], y_train_cat[start:end]))

# -------------------------------
# Build model function
# -------------------------------
def build_model(input_dim, output_dim):
    model = Sequential([
        Dense(256, input_shape=(input_dim,)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.4),

        Dense(128),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),

        Dense(64),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(0.2),

        Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# -------------------------------
# FedAvg Aggregation Function
# -------------------------------
def fedavg(global_model, client_models, client_sizes):
    global_weights = []
    total_samples = np.sum(client_sizes)

    for weights_list_tuple in zip(*[m.get_weights() for m in client_models]):
        weighted_sum = np.sum([w * (n / total_samples) for w, n in zip(weights_list_tuple, client_sizes)], axis=0)
        global_weights.append(weighted_sum)

    global_model.set_weights(global_weights)
    return global_model

# -------------------------------
# Federated Learning Simulation
# -------------------------------
global_model = build_model(X_train.shape[1], y_train_cat.shape[1])
num_rounds = 10
local_epochs = 5

global_acc, client_acc_history = [], []

for round_num in range(num_rounds):
    print(f"\n🌀 Round {round_num + 1}/{num_rounds}")
    client_models = []
    client_sizes = []
    round_accuracies = []

    for client_id, (X_c, y_c) in enumerate(client_data):
        print(f"  - Training Client {client_id + 1}")
        client_model = clone_model(global_model)
        client_model.set_weights(global_model.get_weights())

        client_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        client_model.fit(X_c, y_c, epochs=local_epochs, batch_size=64, verbose=0)

        acc = client_model.evaluate(X_c, y_c, verbose=0)[1]
        round_accuracies.append(acc)
        client_models.append(client_model)
        client_sizes.append(len(X_c))

    # FedAvg aggregation
    global_model = fedavg(global_model, client_models, client_sizes)
    global_eval = global_model.evaluate(X_test, y_test_cat, verbose=0)
    global_acc.append(global_eval[1])
    client_acc_history.append(round_accuracies)

    print(f"✅ Global Accuracy after round {round_num + 1}: {global_eval[1]:.4f}")

# -------------------------------
# Final Evaluation
# -------------------------------
y_pred = np.argmax(global_model.predict(X_test), axis=1)
acc = accuracy_score(y_test, y_pred)
print("\n🎯 Final Federated Model Accuracy:", round(acc, 4))

# -------------------------------
# Visualization
# -------------------------------

# 1️⃣ Global Accuracy across rounds
plt.figure(figsize=(8,5))
plt.plot(range(1, num_rounds+1), global_acc, marker='o')
plt.title("Global Model Accuracy per Round (FedAvg)")
plt.xlabel("Round")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()

# 2️⃣ Client Accuracy trends
plt.figure(figsize=(10,6))
for i in range(num_clients):
    plt.plot(range(1, num_rounds+1), [accs[i] for accs in client_acc_history], label=f'Client {i+1}')
plt.title("Client Training Accuracy per Round")
plt.xlabel("Round")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# 3️⃣ Comparison with Centralized Model
centralized_acc = 0.71  # From your previous model
plt.figure(figsize=(6,5))
plt.bar(['Centralized', 'Federated'], [centralized_acc, acc], color=['skyblue', 'orange'])
plt.title("Accuracy Comparison")
plt.ylabel("Accuracy")
for i, v in enumerate([centralized_acc, acc]):
    plt.text(i, v + 0.01, f"{v:.2f}", ha='center', fontsize=12)
plt.show()


# **ML Model**

**Comparison of ML Models**

In [ ]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# -------------------------------
# 1. Load & clean dataset
# -------------------------------
df = pd.read_csv("/content/SecureCare_Data.csv")
df = df.drop(['Time', 'Person'], axis=1)

# Label encode class names
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])

# -------------------------------
# 2. Window-based feature extraction
# -------------------------------
WINDOW_SIZE = 128
STEP_SIZE = 64

def extract_features(df, window_size=WINDOW_SIZE, step_size=STEP_SIZE):
    feats, labels = [], []
    for start in range(0, len(df) - window_size, step_size):
        end = start + window_size
        window = df.iloc[start:end]
        label = window['Class'].mode()[0]
        feature_dict = {
            'Acc_x_mean': window['Acc_x'].mean(),
            'Acc_y_mean': window['Acc_y'].mean(),
            'Acc_z_mean': window['Acc_z'].mean(),
            'Acc_x_std': window['Acc_x'].std(),
            'Acc_y_std': window['Acc_y'].std(),
            'Acc_z_std': window['Acc_z'].std(),
            'Acc_x_min': window['Acc_x'].min(),
            'Acc_y_min': window['Acc_y'].min(),
            'Acc_z_min': window['Acc_z'].min(),
            'Acc_x_max': window['Acc_x'].max(),
            'Acc_y_max': window['Acc_y'].max(),
            'Acc_z_max': window['Acc_z'].max(),
            'Acc_mag_mean': np.sqrt((window['Acc_x']**2 + window['Acc_y']**2 + window['Acc_z']**2)).mean(),
            'Acc_mag_std': np.sqrt((window['Acc_x']**2 + window['Acc_y']**2 + window['Acc_z']**2)).std(),
            'Acc_range': window[['Acc_x','Acc_y','Acc_z']].max().mean() - window[['Acc_x','Acc_y','Acc_z']].min().mean(),
            'Acc_energy': np.sum(window[['Acc_x','Acc_y','Acc_z']]**2).sum()
        }
        feats.append(feature_dict)
        labels.append(label)
    features_df = pd.DataFrame(feats)
    features_df['Class'] = labels
    return features_df

df_feat = extract_features(df)
print(" Features extracted:", df_feat.shape)

# -------------------------------
# 3. Split dataset
# -------------------------------
X = df_feat.drop('Class', axis=1)
y = df_feat['Class']

scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, test_size=0.2, random_state=42, stratify=y
)

# -------------------------------
# 4. XGBoost hyperparameter tuning
# -------------------------------
xgb_model = XGBClassifier(eval_metric='mlogloss', random_state=42)

param_dist = {
    'n_estimators': [200, 400, 600, 800, 1000],
    'max_depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'reg_lambda': [1, 1.5, 2],
    'reg_alpha': [0, 0.1, 0.2]
}

rand_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist,
    n_iter=15,
    scoring='accuracy',
    n_jobs=-1,
    cv=3,
    verbose=1,
    random_state=42
)

rand_search.fit(X_train, y_train)
best_xgb = rand_search.best_estimator_
print("\n Best XGBoost Parameters:", rand_search.best_params_)

# -------------------------------
# 5. Random Forest model
# -------------------------------
rf_model = RandomForestClassifier(
    n_estimators=300, max_depth=12, random_state=42
)
rf_model.fit(X_train, y_train)

# -------------------------------
# 6. Voting Ensemble (XGB + RF)
# -------------------------------
voting = VotingClassifier(
    estimators=[
        ('xgb', best_xgb),
        ('rf', rf_model)
    ],
    voting='soft'
)
voting.fit(X_train, y_train)

# -------------------------------
# 7. Evaluate all models
# -------------------------------
models = {
    "XGBoost": best_xgb,
    "Random Forest": rf_model,
    "Voting Ensemble": voting
}

results = []
for name, model in models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    results.append((name, acc, prec, rec, f1))
    print(f"{name}: Accuracy={acc:.4f}, Precision={prec:.4f}, Recall={rec:.4f}, F1={f1:.4f}")

# -------------------------------
# 8. Plot performance
# -------------------------------
results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"])
results_df = results_df.sort_values(by="Accuracy", ascending=False)

plt.figure(figsize=(9,6))
plt.barh(results_df["Model"], results_df["Accuracy"], color='skyblue')
plt.xlabel("Accuracy")
plt.title("⚙️ Model Comparison on SecureCare (Windowed Features)")
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

print("\n📊 Final Model Performance Summary:")
print(results_df.to_string(index=False))


**Voting Classifier + FL**

In [ ]:
# ===========================================================
# 🏥 SecureCare HAR — Federated Learning (FedAvg Probabilities)
# Model: Voting Ensemble (Tuned XGBoost + RandomForest)
# Clients: 5 (Stratified splits) | Rounds: 5
# ===========================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
import itertools
import warnings
warnings.filterwarnings("ignore")

# -------------------------------
# 1) Load & clean dataset
# -------------------------------
df = pd.read_csv("/content/SecureCare_Data.csv")
df = df.drop(['Time', 'Person'], axis=1)

# Label encode class names
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])

# -------------------------------
# 2) Window-based feature extraction (HAR-style)
# -------------------------------
WINDOW_SIZE = 128
STEP_SIZE   = 64

def extract_features(df, window_size=WINDOW_SIZE, step_size=STEP_SIZE):
    feats, labels = [], []
    for start in range(0, len(df) - window_size, step_size):
        end = start + window_size
        window = df.iloc[start:end]
        label = window['Class'].mode()[0]
        # Statistical + magnitude features
        ax, ay, az = window['Acc_x'], window['Acc_y'], window['Acc_z']
        amag = np.sqrt(ax**2 + ay**2 + az**2)
        feature_dict = {
            'Acc_x_mean': ax.mean(),   'Acc_y_mean': ay.mean(),   'Acc_z_mean': az.mean(),
            'Acc_x_std':  ax.std(),    'Acc_y_std':  ay.std(),    'Acc_z_std':  az.std(),
            'Acc_x_min':  ax.min(),    'Acc_y_min':  ay.min(),    'Acc_z_min':  az.min(),
            'Acc_x_max':  ax.max(),    'Acc_y_max':  ay.max(),    'Acc_z_max':  az.max(),
            'Acc_mag_mean': amag.mean(),
            'Acc_mag_std':  amag.std(),
            'Acc_range': (pd.DataFrame({'x':ax,'y':ay,'z':az}).max().mean()
                         -pd.DataFrame({'x':ax,'y':ay,'z':az}).min().mean()),
            'Acc_energy':  (ax**2 + ay**2 + az**2).sum()
        }
        feats.append(feature_dict)
        labels.append(label)
    features_df = pd.DataFrame(feats)
    features_df['Class'] = labels
    return features_df

df_feat = extract_features(df)
print(" Features extracted:", df_feat.shape)

# -------------------------------
# 3) Train/Test split + normalization
# -------------------------------
X = df_feat.drop('Class', axis=1)
y = df_feat['Class']

scaler = MinMaxScaler()
Xn = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    Xn, y, test_size=0.2, random_state=42, stratify=y
)

n_classes = len(np.unique(y_train))

# -------------------------------
# 4) Tune XGBoost on global train (same as your best run)
# -------------------------------
xgb_base = XGBClassifier(eval_metric='mlogloss', random_state=42)

param_dist = {
    'n_estimators':    [200, 400, 600, 800, 1000],
    'max_depth':       [4, 6, 8, 10],
    'learning_rate':   [0.01, 0.05, 0.1],
    'subsample':       [0.7, 0.8, 1.0],
    'colsample_bytree':[0.7, 0.8, 1.0],
    'min_child_weight':[1, 3, 5],
    'gamma':           [0, 0.1, 0.2],
    'reg_lambda':      [1, 1.5, 2],
    'reg_alpha':       [0, 0.1, 0.2]
}

search = RandomizedSearchCV(
    xgb_base, param_distributions=param_dist,
    n_iter=15, cv=3, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42
)
search.fit(X_train, y_train)
best_xgb = search.best_estimator_
print("\n Best XGBoost Params:", search.best_params_)

# Fixed RF to pair with tuned XGB in the ensemble
rf_global = RandomForestClassifier(n_estimators=300, max_depth=12, random_state=42)

def build_best_voter():
    return VotingClassifier(
        estimators=[('xgb', best_xgb), ('rf', rf_global)],
        voting='soft'
    )

# -------------------------------
# 5) Helper: evaluate metrics
# -------------------------------
def evaluate(y_true, y_pred):
    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted')
    rec  = recall_score(y_true, y_pred, average='weighted')
    f1   = f1_score(y_true, y_pred, average='weighted')
    return acc, prec, rec, f1

# -------------------------------
# 6) Make 5 stratified clients from the TRAIN set
# -------------------------------
def make_clients(X, y, num_clients=5, seed=42):
    skf = StratifiedKFold(n_splits=num_clients, shuffle=True, random_state=seed)
    splits = []
    for _, idx in skf.split(X, y):
        splits.append(idx)
    clients = []
    for idx in splits:
        clients.append((X[idx], y.iloc[idx].values))
    return clients

clients = make_clients(X_train, y_train, num_clients=5, seed=42)
for i, (Xc, yc) in enumerate(clients, 1):
    print(f" Client {i}: {len(yc)} samples")

# -------------------------------
# 7) Federated training (FedAvg on probabilities)
# -------------------------------
ROUNDS = 30
global_metrics = []
round_client_metrics = []  # list of dicts per round

for r in range(1, ROUNDS+1):
    print(f"\n Federated Round {r}/{ROUNDS}")
    client_probs = []     # predicted probabilities on global test, per client
    client_weights = []   # weight by client size
    client_stats = []     # local metrics on client-held-out validation (optional quick split)

    # Train each client locally on its partition
    for ci, (Xc, yc) in enumerate(clients, 1):
        # simple local validation split (10%) to report client-side metrics (no leakage)
        msk = np.random.RandomState(1000 + r + ci).rand(len(yc)) < 0.9
        Xc_tr, yc_tr = Xc[msk], yc[msk]
        Xc_va, yc_va = Xc[~msk], yc[~msk]

        model = build_best_voter()
        model.fit(Xc_tr, yc_tr)

        # metrics on client's local holdout
        if len(yc_va) > 0:
            yva_pred = model.predict(Xc_va)
            cacc, cprec, crec, cf1 = evaluate(yc_va, yva_pred)
        else:
            cacc = cprec = crec = cf1 = np.nan

        client_stats.append({
            "client": ci,
            "n_train": len(yc_tr),
            "n_val": len(yc_va),
            "acc": cacc, "prec": cprec, "rec": crec, "f1": cf1
        })

        # store probs on the shared global test set for FedAvg
        p_test = model.predict_proba(X_test)  # shape: [n_test, n_classes]
        client_probs.append(p_test)
        client_weights.append(len(yc_tr))  # size-weighted FedAvg

    # FedAvg on probabilities (size-weighted)
    client_weights = np.array(client_weights, dtype=float)
    client_weights = client_weights / client_weights.sum()
    P = np.zeros_like(client_probs[0])
    for w, P_i in zip(client_weights, client_probs):
        P += w * P_i

    y_pred_global = np.argmax(P, axis=1)
    acc, prec, rec, f1 = evaluate(y_test, y_pred_global)
    global_metrics.append((acc, prec, rec, f1))
    round_client_metrics.append(client_stats)

    print(f" Global after Round {r}: "
          f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | F1={f1:.4f}")

# -------------------------------
# 8) Plot global metrics per round
# -------------------------------
gm = np.array(global_metrics)
r = np.arange(1, ROUNDS+1)

plt.figure(figsize=(10,5))
plt.plot(r, gm[:,0], marker='o', label='Accuracy')
plt.plot(r, gm[:,1], marker='o', label='Precision (weighted)')
plt.plot(r, gm[:,2], marker='o', label='Recall (weighted)')
plt.plot(r, gm[:,3], marker='o', label='F1 (weighted)')
plt.xlabel("Federated Round")
plt.ylabel("Score")
plt.title("Global Performance per Federated Round (Voting Ensemble)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.show()

# -------------------------------
# 9) Final confusion matrix (last round)
# -------------------------------
cm = confusion_matrix(y_test, y_pred_global, labels=np.arange(n_classes))

plt.figure(figsize=(6,5))
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix (Global Model, Final Round)")
plt.colorbar()
tick_marks = np.arange(n_classes)
plt.xticks(tick_marks, le.inverse_transform(np.arange(n_classes)), rotation=45, ha='right')
plt.yticks(tick_marks, le.inverse_transform(np.arange(n_classes)))
plt.ylabel('True label')
plt.xlabel('Predicted label')

# annotate counts
thresh = cm.max() / 2.0
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
plt.tight_layout()
plt.show()

# -------------------------------
# 10) Print per-client stats (last round)
# -------------------------------
print("\n Per-Client Metrics (last round local holdout):")
for s in round_client_metrics[-1]:
    print(f"Client {s['client']}: n_train={s['n_train']}, n_val={s['n_val']}, "
          f"acc={s['acc']:.4f}, prec={s['prec']:.4f}, rec={s['rec']:.4f}, f1={s['f1']:.4f}")

# -------------------------------
# 11) Print global summary
# -------------------------------
print("\n Global Metrics per Round (Acc, Prec, Rec, F1):")
for i, m in enumerate(global_metrics, 1):
    print(f"Round {i}: Acc={m[0]:.4f}, Prec={m[1]:.4f}, Rec={m[2]:.4f}, F1={m[3]:.4f}")
